<a id='functioncalling'></a>
### 🧪 Test OpenAI function calling

The following code was reused from the [Azure OpenAI documentation](https://learn.microsoft.com/azure/ai-services/openai/how-to/function-calling?tabs=non-streaming%2Cpython). The `UUID` will be used to search for the end-to-end transaction in Application insights.

- Note: run ```pip install openai``` in a terminal before executing this step.

In [ ]:
# type: ignore
from openai import AzureOpenAI
import uuid

prompt = "What's the weather like in London and its big sister cities?"

UUID = str(uuid.uuid4())
print(
    f"Request-Id: {UUID} - use this ID to trace the requests in Azure Application Insights."
)

client = AzureOpenAI(
    azure_endpoint=apim_resource_gateway_url,
    api_key=apim_subscription_key,
    api_version=openai_api_version,
)


# Example function hard coded to return the same weather
# the local function calls the API
def get_current_weather(location, unit):
    request = {"location": location, "unit": unit}
    url = apim_resource_gateway_url + "/weather"
    response = requests.post(
        url,
        headers={"api-key": apim_subscription_key, "Request-Id": UUID},
        json=request,
    )
    if response.status_code == 200:
        return response.text
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


# Step 1: send the conversation and available functions to the model
messages = [{"role": "user", "content": prompt}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location, using the local temperature measuring unit.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "description": "The temperature measuring unit, e.g. celsius for London or fahrenheit for US cities",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location", "unit"],
            },
        },
    }
]
print(
    "▶️ Step 1: start a completion to identify the appropriate functions to invoke based on the prompt\n",
    prompt,
)
response = client.chat.completions.create(
    model=openai_deployment_name,
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
    extra_headers={"Request-Id": UUID},
)
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
if tool_calls:
    # Step 2: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
    }  # only one function in this example, but you can have multiple
    messages.append(response_message)  # extend conversation with assistant's reply
    # send the info for each function call and function response to the model
    print("▶️ Step 2: call the functions")
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        print(function_response)
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
    print(
        "▶️ Step 3: finish with a completion to anwser the user prompt using the function response"
    )
    second_response = client.chat.completions.create(
        model=openai_deployment_name,
        messages=messages,
        extra_headers={"Request-Id": UUID},
    )  # get a new response from the model where it can see the function response
    print(second_response.choices[0].message.content)